In [3]:
from datetime import datetime
from zoneinfo import ZoneInfo

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Dropdown

import sys
sys.path.append("src")

from energy_es.data.prices import PricesManager

In [4]:
# Price unit
unit_wid = Dropdown(options=[("€/KWh", "k"), ("€/MWh", "m")], value="k")
unit_wid

Dropdown(options=(('€/KWh', 'k'), ('€/MWh', 'm')), value='k')

In [5]:
unit = unit_wid.value
unit_label = unit_wid.label

print("Price unit selected:", unit_label)

Price unit selected: €/KWh


# Get data

In [6]:
pm = PricesManager()
prices = pm.get_prices(unit)

updated = prices["updated"]
price_unit = prices["price_unit"]
data = prices["data"]

print(updated)
print(price_unit)
print(data[0])  # Sample

1671150803.850847
€/KWh
{'time': '00:00', 'spot_market': 0.13, 'pvpc_pcb': 0.22596, 'pvpc_cm': 0.22596}


# Title and source

In [7]:
dt = datetime.fromtimestamp(updated).astimezone(ZoneInfo("Europe/Madrid"))
dt = dt.strftime(f"%A {dt.day} %B %Y (%Z)")

title = f"Electricity price ({price_unit}) in Spain for {dt}"
source = "Data source: Red Eléctrica de España"

print(title)
print(source)

Electricity price (€/KWh) in Spain for Friday 16 December 2022 (CET)
Data source: Red Eléctrica de España


# Create dataframe

In [8]:
prices_df = pd.DataFrame(data)

text = ["<b>MIN</b>", "<b>MAX</b>"]
text_pos = ["bottom center", "top center"]

for c in ("spot_market", "pvpc_pcb", "pvpc_cm"):
    cond = [
        prices_df[c] == prices_df[c].min(),
        prices_df[c] == prices_df[c].max()
    ]
    
    prices_df[f"{c}_text"] = np.select(cond, text, default=None)

    prices_df[f"{c}_text_position"] = (
        np.select(cond, text_pos, default="top center")
    )

prices_df

,time,spot_market,pvpc_pcb,pvpc_cm,spot_market_text,spot_market_text_position,pvpc_pcb_text,pvpc_pcb_text_position,pvpc_cm_text,pvpc_cm_text_position
0,00:00,0.13000,0.22596,0.22596,None,top center,None,top center,None,top center
1,01:00,0.12274,0.22066,0.22066,None,top center,None,top center,None,top center
2,02:00,0.11200,0.21213,0.21213,None,top center,None,top center,None,top center
3,03:00,0.10368,0.20442,0.20442,None,top center,None,top center,None,top center
4,04:00,0.10100,0.19963,0.19963,<b>MIN</b>,bottom center,<b>MIN</b>,bottom center,<b>MIN</b>,bottom center
5,05:00,0.10368,0.20195,0.20195,None,top center,None,top center,None,top center
6,06:00,0.11101,0.20344,0.20344,None,top center,None,top center,None,top center
7,07:00,0.12208,0.20205,0.20205,None,top center,None,top center,None,top center
8,08:00,0.14000,0.24298,0.24298,None,top center,None,top center,None,top center
9,09:00,0.13482,0.23432,0.23432,None,top center,None,top center,None,top center


# Create chart

In [19]:
fig = go.Figure()

fig.update_layout(
    title={
        "text":
            f'{title}<br><span style="font-size: 14px">{source}</span>',
        "yref": "paper",
        "y": 1,
        "yanchor": "bottom",
        "pad": {"l": 77, "b": 40},
        "x": 0,
        "xanchor": "left"
    },
    plot_bgcolor="white",
    xaxis={
        "title": "Time",
        "fixedrange": True,
        "showline": True,
        "mirror": True,
        "linecolor": "black",
        "gridcolor": "lightgrey",
        "ticks": "outside",
        "tickangle": 45
    },
    yaxis={
        "title": price_unit,
        "fixedrange": True,
        "showline": True,
        "mirror": True,
        "linecolor": "black",
        "gridcolor": "lightgrey",
        "ticks": "outside"
    },
    legend={"itemdoubleclick": False},
    margin={"t": 65}
)

hover_tem = "Time: &nbsp;%{x}<br>Price: &nbsp;%{y} " + price_unit
spot_hover_tem = "<b>Spot Market</b><br>" + hover_tem

pvpc_pcb_hover_tem = (
    "<b>PVPC (Peninsula, Canarias and Baleares)</b><br>" + hover_tem
)

pvpc_cm_hover_tem = "<b>PVPC (Ceuta and Melilla)</b><br>" + hover_tem

# Spot market prices
spot_sca = go.Scatter(
    x=prices_df["time"],
    y=prices_df["spot_market"],
    mode="lines+markers+text",
    text=prices_df["spot_market_text"],
    line={"width": 3, "color": "#2077b4"},
    marker={"size": 12, "color": "#2077b4"},
    textposition=prices_df["spot_market_text_position"],
    textfont={"color": "#2077b4"},
    name="Spot Market price",
    hovertemplate=spot_hover_tem,
    hoverlabel={"namelength": 0}
)

# PVPC prices (Peninsula, Canarias and Baleares)
pvpc_pcb_sca = go.Scatter(
    x=prices_df["time"],
    y=prices_df["pvpc_pcb"],
    mode="lines+markers+text",
    text=prices_df["pvpc_pcb_text"],
    line={"width": 3, "color": "#ff8c00"},
    marker={"size": 12, "color": "#ff8c00"},
    textposition=prices_df["pvpc_pcb_text_position"],
    textfont={"color": "#ff8c00"},
    name="PVPC price<br>(Peninsula, Canarias<br>and Baleares)",
    hovertemplate=pvpc_pcb_hover_tem,
    hoverlabel={"namelength": 0}
)

# PVPC prices (Ceuta and Melilla)
pvpc_cm_sca = go.Scatter(
    x=prices_df["time"],
    y=prices_df["pvpc_cm"],
    mode="lines+markers+text",
    text=prices_df["pvpc_cm_text"],
    line={"width": 3, "color": "#00a002"},
    marker={"size": 12, "color": "#00a002"},
    textposition=prices_df["pvpc_cm_text_position"],
    textfont={"color": "#00a002"},
    name="<br>PVPC price<br>(Ceuta and Melilla)<br>",
    hovertemplate=pvpc_cm_hover_tem,
    hoverlabel={"namelength": 0}
)

fig.add_traces([spot_sca, pvpc_cm_sca, pvpc_pcb_sca])
conf = {"displayModeBar": False}

fig.show(config=conf)